<a href="https://colab.research.google.com/github/saurabhkirar/EPAI01/blob/main/Capstone/Codes/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## start of the project.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import cv2
import smtplib
from PIL import Image, ImageDraw, ImageFont
from datetime import date
import os
import validation
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
import os
project_path='/content/drive/MyDrive/EPAI/Capstone/Codes/'
image_path='/content/drive/MyDrive/EPAI/Capstone/Data/'
out_path='/content/drive/MyDrive/EPAI/Capstone/Out/'
Image_name=image_path+'Cert.jpg'
sender_email='souravkirar@gmail.com'
subject = "Congratutations on course completion!!"
os.chdir(project_path)

In [ ]:
## Class to load the data from the csv file.

In [ ]:
def create_or_locate_cert(p_course,p_name,p_recreate=False):

    '''This functions creates the certificate for a given user and the course 
     and stores the certificates in the out directory
     If for a user and the course the certificate already exists then the function fectes the certificate 
     from the out directory and returns the cert, if not then fuction goes on to make a new certificate
     '''
    course = p_course
    name = p_name
    color = 'rgb(0, 0, 0)' # black color
       # if exists and overwrite is False simply return with certificate path
    img=out_path+name+course+'cert.jpg'
    if (not p_recreate) and (os.path.exists(img)):
            print(f'Certificate already exists,resending')
            return img

    # create Image object with the input image
    image = Image.open(Image_name)
    # initialise the drawing context with the image object as background
    draw = ImageDraw.Draw(image)
    # create font object with the font file and specify desired size   
    font_course = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf", size=20)
    # starting position of the message
    (x, y) = (450, 270)
    # draw the message on the background
    draw.text((x, y), course, fill=color,font=font_course)

    font_name = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf", size=30)
    (x, y) = (450, 420)
    draw.text((x, y), name, fill=color,font=font_name)


    font_date = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf", size=30)
    (x, y) = (235, 500)
    today = date.today()
    today=str(today)
    draw.text((x, y), today, fill=color,font=font_name)

    font_sign = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf", size=30)
    (x, y) = (650, 500)
    admin_name = 'School of AI'
    draw.text((x, y), admin_name, fill=color,font=font_name)
    
    # save the edited image
    img=out_path+name+course+'cert.jpg'

    image.save(img)

    #I=cv2.imread(img)
    #plt.imshow(I)
    
    return img

In [ ]:
@validation.logging_decorator()
@validation.internet_check_decorator()
#@validation.is_authenticated(p_user_name,)
def send_certificate(p_receiver_email,p_key,p_user_name,p_course_name,p_score,p_total,p_recreate=False):
  '''This is a function which will send the certificate and can be called with the below params
  p_receiver_email-> Receiver email address.
  p_key-> Password for email account (sender)
  p_user-> User to whom the certificate should go
  p_course_name-> the name of the course for which the certs are being awarded
  p_score/p_total -> marks obtained/total marks
  p_recreate->Re create the certficiate even if it exists(chances of correction)
  '''
  receiver_email = p_receiver_email
  password = p_key
     # Create a multipart message and set headers
  message = MIMEMultipart()
  message["From"] = sender_email
  message["To"] = receiver_email
  message["Subject"] = subject
  message["Bcc"] = receiver_email  # Recommended for mass emails

  body='''
        Dear {name},

        Congratulations!

        You have cleared the {course_name} with {score} marks out of {total} marks!
        We are excited to share the attached Award of Excellence for your performance!

        Regards
        '''.format(name=p_user_name,course_name=p_course_name,score=p_score,total=p_total)

 # Add body to email
  message.attach(MIMEText(body, "plain"))

  #filename = "BroadBandAug20.pdf"  # In same directory as script
  filename=create_or_locate_cert(p_course_name,p_user_name,p_recreate)

  with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

  # Encode file in ASCII characters to send by email    
  encoders.encode_base64(part)

  # Add header as key/value pair to attachment part
  part.add_header(
      "Content-Disposition",
      f"attachment; filename= {filename}",
  )

  # Add attachment to message and convert message to string
  message.attach(part)
  text = message.as_string()

  # Log in to server using secure context and send email
  context = ssl.create_default_context()
  with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, text)

In [ ]:
send_certificate('saurabh.kirar09@gmail.com','Pwd@gmailold2210','Saurabh Kirar','EPAI',98,100)

Certificate already exists,resending
